In [5]:
import numpy as np
import plotly.express as px

import sys
sys.path.append("../")

%load_ext autoreload
%autoreload 2

from IMLearn.learners import gaussian_estimators as ge
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Univariate

In [310]:
# q1
underlying_mean = 10
underlying_var = 1
sample_size = 100

X = np.random.normal(underlying_mean, underlying_var, sample_size)
ug = ge.UnivariateGaussian() # TODO: biased or not?
ug.fit(X)
print(f"({ug.mu_}, {ug.var_})")

(9.986647030137876, 0.785148501298852)


In [312]:
sizes = np.arange(1, 101) * 10
expectations = np.empty_like(sizes, dtype=np.float32)

for i, size in enumerate(sizes):
    ug = ge.UnivariateGaussian()
    ug.fit(X[:size])
    expectations[i] = ug.mu_
    
abs_distance = np.abs(expectations - underlying_mean)
title = "absolute distance between estimates mean and true mean, as function of sample size (showing consistency)"
figure = px.scatter(x=sizes, y=abs_distance, labels={"x": "sample size", "y": "abs distance between estimated & true mean"}, title=title)
figure.show()


In [316]:
# q3:
# I would expect to see the shape of the normal distribution PDF, with mean ±10, var ±1. That's because the estimator estimates the mean and variance quite well.
# Additionally, as it is more likely to draw samples from around the mean, there will probably be more samples with height of around f(10), e.g., ±0.4.
# That's because the probability of drawing samples from that area is higher than in the tails

ug = ge.UnivariateGaussian()
ug.fit(X)
figure = px.scatter(x=X, y=ug.pdf(X), labels={"x": "sample value", "y": "estimated pdf value of sample"}, title="samples to the estimated gaussian pdf value")
figure.show()


## Multivariate

In [139]:
# q4
mu = np.array([0, 0, 4, 0])
cov_matrix = np.array([
    [1.0, 0.2, 0.0, 0.5],
    [0.2, 2.0, 0.0, 0.0],
    [0.0, 0.0, 1.0, 0.0],
    [0.5, 0.0, 0.0, 1.0],
])
samples = np.random.multivariate_normal(mu, cov_matrix, size=1000)
assert samples.shape == (1000, 4)

mv_ge = ge.MultivariateGaussian()
mv_ge.fit(samples)

print(mv_ge.mu_)
print(mv_ge.cov_)


[ 0.022652   -0.03454315  3.98705152 -0.01540247]
[[ 1.03527307  0.27761977  0.0286019   0.54800036]
 [ 0.27761977  1.92646258 -0.02654667  0.07353079]
 [ 0.0286019  -0.02654667  0.92132152  0.00402026]
 [ 0.54800036  0.07353079  0.00402026  1.02750776]]


In [115]:
# q5
import tqdm

f_space = np.linspace(-10, 10, 200)
log_likelihoods = np.empty((len(f_space), len(f_space)))
for i, f1 in enumerate(tqdm.tqdm(f_space)):
    for j, f3 in enumerate(f_space):
        log_likelihoods[i, j] = ge.MultivariateGaussian.log_likelihood(np.array([f1, 0, f3, 0]), cov_matrix, samples)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [04:16<00:00,  1.28s/it]


In [319]:
"""
what are you able to learn from the plot?
The plot can be interpreted similarily to a countour plot, 
in the sense that the LL, as a function of (f1, f3), recieves higher values in the (f1, f3) values corresponding to the yellow ellispoides. 
this means that in order to maximize the LL, we would prefer using a 'mu' with correspondance to those values.
"""

title = "Log-likelihood heatmap, with correspondance to the different f1 & f3 values (different mean)"
px.imshow(log_likelihoods, title=title, labels={"x": "f3", "y": "f1"}).show()





"\nwhat are you able to learn from the plot?\nThe plot can be interpreted similarily to a countour plot, \nin the sense that the LL, as a function of (f1, f3), recieves higher values in the (f1, f3) values corresponding to the yellow ellispoides. \nthis means that in order to maximize the LL, we would prefer using a 'mu' with correspondance to those values.\n"

In [320]:
# q6
round_to = 3
i, j = np.unravel_index(np.argmax(log_likelihoods), log_likelihoods.shape)
f1 = f_space[i].round(round_to)
f2 = f_space[j].round(round_to)
f1, f2
# TODO: make sure rounding to 3

(-0.05, 3.97)

## Tests

In [307]:
import numpy as np
import plotly.express as px

import sys
sys.path.append("../")

from IMLearn.learners import gaussian_estimators as ge
from utils import *
from scipy.stats import multivariate_normal
import math


print("uni.fit")
max_diff = 3
mu = 5; std = 10; m = 100000
samples = np.random.normal(loc=mu, scale=std, size=m)
fitted = ge.UnivariateGaussian(biased_var=False).fit(samples)
fitted_biased = ge.UnivariateGaussian(biased_var=True).fit(samples)
print(f"expected mu: {mu}")
print(f"our mu:      {fitted.mu_}\n")
print(f"expected var: {std ** 2}")
print(f"our var:      {fitted.var_}\n")

assert isinstance(fitted.mu_, float)
assert isinstance(fitted.var_, float)
assert isinstance(fitted_biased.mu_, float)
assert isinstance(fitted_biased.var_, float)

assert np.all(np.abs(fitted.mu_ - mu) < max_diff)
assert np.allclose(fitted_biased.mu_, fitted.mu_)
assert np.all(np.abs(fitted_biased.var_ - std ** 2) < max_diff)
assert(fitted_biased.var_ / (m - 1) == fitted.var_ / m)


print("uni.pdf")
max_diff = 0.001
mu = 5; std = 2
fitted = ge.UnivariateGaussian(biased_var=False).fit(np.random.normal(loc=mu, scale=std, size=10000000))
x = mu + np.random.rand(1000)
expected = multivariate_normal.pdf(x, mean=mu, cov=std ** 2);
our = fitted.pdf(x)
print(f"expected: {expected[:5]}")
print(f"our:      {our[:5]}\n")
assert np.all(np.abs(expected - our) < max_diff)
assert our.shape == x.shape

print("uni.log_likelihood")
x = mu + np.random.rand(10)
expected = np.log(np.prod(multivariate_normal.pdf(x, mean=mu, cov=std ** 2)))
our = fitted.log_likelihood(mu, std ** 2, x)
print(f"expected: {expected}")
print(f"our:      {our}\n")
assert np.allclose(expected, our)
assert isinstance(our, float)


print("multi.fit")
max_diff = 0.3
mu = np.array([5, -15]);
cov = np.array([
    [10, 3],
    [3,  40],
])
m = 1000000

samples = np.random.multivariate_normal(mu, cov, size=m)
fitted = ge.MultivariateGaussian().fit(samples)
print(f"expected mu:  {mu}")
print(f"our mu:       {fitted.mu_}\n")
print(f"expected var: \n {cov}")
print(f"our var:      \n {fitted.cov_}\n") # TODO: should be biased or not

assert np.all(np.abs(fitted.mu_ - mu) < max_diff)
assert np.all(np.abs(fitted.cov_ - cov) < max_diff)
assert fitted.mu_.shape == mu.shape
assert fitted.cov_.shape == cov.shape


print("multi.pdf")
x = mu + np.random.rand(10, 2)
expected = multivariate_normal.pdf(x, mean=mu, cov=cov);
our = fitted.pdf(x)
print(f"expected: {expected[:5]}")
print(f"our:      {our[:5]}\n")
assert np.all(np.abs(expected - our) < max_diff)
assert our.shape == (len(x),)


print("multi.log_likelihood")
expected = np.log(np.prod(multivariate_normal.pdf(x, mean=mu, cov=cov)))
our = fitted.log_likelihood(mu, cov, x)
print(f"expected: {expected}")
print(f"our:      {our}")
assert np.allclose(expected, our)
assert isinstance(our, float)

print("\nsuccess.")


uni.fit
expected mu: 5
our mu:      5.0383406189251705

expected var: 100
our var:      99.4497113673277

uni.pdf
expected: [0.1798458  0.18507926 0.18212076 0.19247288 0.19946451]
our:      [0.17990747 0.18514514 0.18218428 0.19254421 0.19953759]

uni.log_likelihood
expected: -16.348448272264356
our:      -16.348448272264353

multi.fit
expected mu:  [  5 -15]
our mu:       [  4.99278113 -14.99823791]

expected var: 
 [[10  3]
 [ 3 40]]
our var:      
 [[ 9.9937725   3.00212366]
 [ 3.00212366 39.92080421]]

multi.pdf
expected: [0.00785185 0.00803918 0.00798172 0.00786993 0.0079255 ]
our:      [0.0078589  0.00804921 0.00799268 0.0078773  0.00793387]

multi.log_likelihood
[0.00785185 0.00803918 0.00798172 0.00786993 0.0079255  0.00787481
 0.00800355 0.0077448  0.00801867 0.00791981]
expected: -48.380472466783765
our:      -48.380472466783765

success.
